In [ ]:
# import all packages
 
import numpy as np

In [ ]:
data = np.loadtxt('DMDdata200.csv', delimiter=',')

In [ ]:
x1 = data[0:,1:-1]
x2 = data[0:,2:]

In [ ]:
data_shape = (x1.shape[1], x1.shape[0])
percent_Trunc = 0.90
Frate = 1/2.5e-05
dt = 2.5e-05

In [ ]:
# Compression matrix generation

C_gaussian = np.random.normal(0, 1, size=data_shape)

In [ ]:
# compress data

y1 = np.dot(C_gaussian, x1)
y2 = np.dot(C_gaussian, x2)

In [ ]:
# cDMD
start = 0
end = 0
import time
start = time.time()

[Uc, Sc, Vc] = np.linalg.svd(y1, full_matrices=False, compute_uv=True, hermitian=False)

menergyc = 0.0
for i in range(0, np.size(Sc)):
    menergyc += Sc[i]/np.sum(Sc)
    if menergyc > percent_Trunc:
        break
    rc = i
    
menergyc, rc

Atildec = Uc[:, :rc].T @ y2 @ Vc[:, :rc] * np.reciprocal(Sc[: rc])
[eivValc,eivVecc] = np.linalg.eig(Atildec)

phic = x2 @ Vc[:rc, :].T @ np.diag(np.reciprocal(Sc[: rc])) @ eivVecc
bc = np.linalg.lstsq(phic, x1[:, 0])

num_modesc = phic.shape[1]
omegac = np.log(eivValc) / dt

end = time.time()
print(end - start)

In [ ]:
## Reconstruct the field

num_stepsc = 2000
time_dynamicsc = np.zeros((num_modesc, num_stepsc), dtype=np.complex128)
for i in range(num_stepsc):
    time_dynamicsc[:, i] = bc[0] * np.exp(omegac * i)

reconstructed_fieldc = phic @ time_dynamicsc

In [ ]:
# DMD

start = 0
end = 0

import time
start = time.time()

[U, S, V] = np.linalg.svd(x1, full_matrices=False, compute_uv=True, hermitian=False)

menergy = 0.0
for i in range(0, np.size(S)):
    menergy += S[i]/np.sum(S)
    if menergy > percent_Trunc:
        break
    r = i
    
menergy, r

Atilde = U[:, :r].T @ x2 @ V[:, :r] * np.reciprocal(S[: r])
[eivVal,eivVec] = np.linalg.eig(Atilde)

phi = x2 @ V[:r, :].T @ np.diag(np.reciprocal(S[: r])) @ eivVec # Compute full-state modes of X
b = np.linalg.lstsq(phi, x1[:, 0])

num_modes = phi.shape[1]
omega = np.log(eivVal) / dt

end = time.time()
print(end - start)

In [ ]:
## Reconstruct the field

num_steps = 2000
time_dynamics = np.zeros((num_modes, num_steps), dtype=np.complex128)
for i in range(num_steps):
    time_dynamics[:, i] = b[0] * np.exp(omega * i)

reconstructed_field = phi @ time_dynamics